# Merging data


## 1. Merging on a specific column
This exercise follows on the last one with the DataFrames revenue and managers for your company. You expect your company to grow and, eventually, to operate in cities with the same name on different states. As such, you decide that every branch should have a numerical branch identifier. Thus, you add a `branch_id` column to both DataFrames. Moreover, new cities have been added to both the revenue and managers DataFrames as well. 

At present, there should be a 1-to-1 relationship between the `city` and `branch_id` fields. In that case, the result of a merge on the city columns ought to give you the same output as a merge on the `branch_id` columns. Do they? Can you spot an ambiguity in one of the DataFrames?

In [1]:
import pandas as pd

In [46]:
# Loading data
city = ["Austin", "Denver", "Springfield", "Mendocino"]
branch_id = [10, 20, 30, 47]
rev = [100, 83, 4, 200]

revenue = pd.DataFrame({"city": city, "branch_id": branch_id, "revenue": rev})

revenue

,city,branch_id,revenue
0,Austin,10,100
1,Denver,20,83
2,Springfield,30,4
3,Mendocino,47,200


In [47]:
# Loading data
city = ["Austin", "Denver", "Mendocino", "Springfield"]
branch_id = [10, 20, 47, 31]
man = ["Charles", "Joel", "Brett", "Sally"]

managers = pd.DataFrame({"city": city, "branch_id": branch_id, "manager": man})

managers

,city,branch_id,manager
0,Austin,10,Charles
1,Denver,20,Joel
2,Mendocino,47,Brett
3,Springfield,31,Sally


In [48]:
# Merge revenue with managers on 'city': merge_by_city
merge_by_city = pd.merge(revenue, managers, on="city")

# Print merge_by_city
merge_by_city

,city,branch_id_x,revenue,branch_id_y,manager
0,Austin,10,100,10,Charles
1,Denver,20,83,20,Joel
2,Springfield,30,4,31,Sally
3,Mendocino,47,200,47,Brett


Well done! Notice that when you merge on `'city'`, the resulting DataFrame has a peculiar result: In row 2, the city Springfield has two different branch IDs. This is because there are actually two different cities named Springfield - one in the State of Illinois, and the other in Missouri. The revenue DataFrame has the one from Illinois, and the managers DataFrame has the one from Missouri. 

In [49]:
# Merge revenue with managers on 'branch_id': merge_by_id
merge_by_id = pd.merge(revenue, managers, on="branch_id")

# Print merge_by_id
merge_by_id

,city_x,branch_id,revenue,city_y,manager
0,Austin,10,100,Austin,Charles
1,Denver,20,83,Denver,Joel
2,Mendocino,47,200,Mendocino,Brett


Consequently, when you merge on `'branch_id'`, both of these get dropped from the merged DataFrame.



## 2. Merging on columns with non-matching labels
You continue working with the revenue & managers DataFrames from before. This time, someone has changed the field name 'city' to 'branch' in the managers table. Now, when you attempt to merge DataFrames, an exception is thrown:

```python
>>> pd.merge(revenue, managers, on='city')
Traceback (most recent call last):
    ... <text deleted> ...
    pd.merge(revenue, managers, on='city')
    ... <text deleted> ...
KeyError: 'city'
```

Given this, it will take a bit more work for you to join or merge on the city/branch name. You have to specify the `left_on` and `right_on` parameters in the call to `pd.merge()`.

Are you able to merge better than in the last exercise? How should the rows with Springfield be handled?

In [50]:
# Preparing dataframes
state = ["TX", "CO", "IL", "CA"]
revenue["state"] = state
revenue

,city,branch_id,revenue,state
0,Austin,10,100,TX
1,Denver,20,83,CO
2,Springfield,30,4,IL
3,Mendocino,47,200,CA


In [51]:
# Preparing dataframes
state = ["TX", "CO", "CA", "MO"]
managers["state"] = state
managers.columns = ["branch", "branch_id", "manager", "state"]
managers

,branch,branch_id,manager,state
0,Austin,10,Charles,TX
1,Denver,20,Joel,CO
2,Mendocino,47,Brett,CA
3,Springfield,31,Sally,MO


In [52]:
# Merge revenue & managers on 'city' & 'branch': combined
combined = pd.merge(revenue, managers, left_on = "city", right_on = "branch")

# Print combined
combined

,city,branch_id_x,revenue,state_x,branch,branch_id_y,manager,state_y
0,Austin,10,100,TX,Austin,10,Charles,TX
1,Denver,20,83,CO,Denver,20,Joel,CO
2,Springfield,30,4,IL,Springfield,31,Sally,MO
3,Mendocino,47,200,CA,Mendocino,47,Brett,CA


Great work! It is important to pay attention to how columns are named in different DataFrames.



## 3. Merging on multiple columns
Another strategy to disambiguate cities with identical names is to add information on the states in which the cities are located. To this end, you add a column called `state` to both DataFrames from the preceding exercises.

Your goal in this exercise is to use `pd.merge()` to merge DataFrames using multiple columns (using `'branch_id'`, `'city'`, and `'state'` in this case).

Are you able to match all your company's branches correctly

In [53]:
# Add 'state' column to revenue: revenue['state']
revenue['state'] = ['TX','CO','IL','CA']

revenue

,city,branch_id,revenue,state
0,Austin,10,100,TX
1,Denver,20,83,CO
2,Springfield,30,4,IL
3,Mendocino,47,200,CA


In [55]:
# Add 'state' column to managers: managers['state']
managers['state'] = ['TX','CO','CA','MO']

managers.columns = ["city", "branch_id", "manager", "state"]

managers

,city,branch_id,manager,state
0,Austin,10,Charles,TX
1,Denver,20,Joel,CO
2,Mendocino,47,Brett,CA
3,Springfield,31,Sally,MO


In [56]:
# Merge revenue & managers on 'branch_id', 'city', & 'state': combined
combined = pd.merge(revenue, managers, on=["branch_id", "state", "city"])

# Print combined
combined

,city,branch_id,revenue,state,manager
0,Austin,10,100,TX,Charles
1,Denver,20,83,CO,Joel
2,Mendocino,47,200,CA,Brett


Excellent work! You've matched all the branches correctly!



## 4. Left & right merging on multiple columns
You now have, in addition to the revenue and managers DataFrames from prior exercises, a DataFrame sales that summarizes units sold from specific branches (identified by city and state but not branch_id).

Once again, the managers DataFrame uses the label branch in place of city as in the other two DataFrames. Your task here is to employ left and right merges to preserve data and identify where data is missing.

By merging revenue and sales with a right merge, you can identify the missing revenue values. Here, you don't need to specify left_on or right_on because the columns to merge on have matching labels.

By merging sales and managers with a left merge, you can identify the missing manager. Here, the columns to merge on have conflicting labels, so you must specify left_on and right_on. In both cases, you're looking to figure out how to connect the fields in rows containing Springfield.

pandas has been imported as pd and the three DataFrames revenue, managers, and sales have been pre-loaded. They have been printed for you to explore in the IPython Shell.

In [62]:
# Print revenue
revenue

,city,branch_id,revenue,state
0,Austin,10,100,TX
1,Denver,20,83,CO
2,Springfield,30,4,IL
3,Mendocino,47,200,CA


In [59]:
# Prepare managers and print it
managers.columns = ["branch", "branch_id", "manager", "state"]
managers

,branch,branch_id,manager,state
0,Austin,10,Charles,TX
1,Denver,20,Joel,CO
2,Mendocino,47,Brett,CA
3,Springfield,31,Sally,MO


In [61]:
# Prepare sales dataframe and print it
city = ["Mendocino", "Denver", "Austin", "Springfield", "Springfield"]
state = ["CA", "CO", "TX", "MO", "IL"]
units = [1,4,2,5,1]

sales = pd.DataFrame({"city": city, "state": state, "units": units})

sales

,city,state,units
0,Mendocino,CA,1
1,Denver,CO,4
2,Austin,TX,2
3,Springfield,MO,5
4,Springfield,IL,1


In [63]:
# Merge revenue and sales: revenue_and_sales
revenue_and_sales = pd.merge(revenue, sales, how="right", on=["city", "state"])

# Print revenue_and_sales
revenue_and_sales

,city,branch_id,revenue,state,units
0,Austin,10.0,100.0,TX,2
1,Denver,20.0,83.0,CO,4
2,Springfield,30.0,4.0,IL,1
3,Mendocino,47.0,200.0,CA,1
4,Springfield,NaN,NaN,MO,5


In [64]:
# Merge sales and managers: sales_and_managers
sales_and_managers = pd.merge(sales, managers, how="left", left_on= ["city", "state"], right_on = ["branch", "state"])

# Print sales_and_managers
sales_and_managers

,city,state,units,branch,branch_id,manager
0,Mendocino,CA,1,Mendocino,47.0,Brett
1,Denver,CO,4,Denver,20.0,Joel
2,Austin,TX,2,Austin,10.0,Charles
3,Springfield,MO,5,Springfield,31.0,Sally
4,Springfield,IL,1,NaN,NaN,NaN


Well done! This is a good way to retain both entries of Springfield.



## 5. Merging DataFrames with outer join
This exercise picks up where the previous one left off. 

The merged DataFrames contain enough information to construct a DataFrame with 5 rows with all known information correctly aligned and each branch listed only once. You will try to merge the merged DataFrames on all matching keys (which computes an inner join by default). You can compare the result to an outer join and also to an outer join with restricted subset of columns as keys.

In [65]:
# Perform the first merge: merge_default
merge_default = pd.merge(sales_and_managers, revenue_and_sales)

# Print merge_default
merge_default

,city,state,units,branch,branch_id,manager,revenue
0,Mendocino,CA,1,Mendocino,47.0,Brett,200.0
1,Denver,CO,4,Denver,20.0,Joel,83.0
2,Austin,TX,2,Austin,10.0,Charles,100.0


In [66]:
# Perform the second merge: merge_outer
merge_outer = pd.merge(sales_and_managers, revenue_and_sales, how="outer")

# Print merge_outer
merge_outer

,city,state,units,branch,branch_id,manager,revenue
0,Mendocino,CA,1,Mendocino,47.0,Brett,200.0
1,Denver,CO,4,Denver,20.0,Joel,83.0
2,Austin,TX,2,Austin,10.0,Charles,100.0
3,Springfield,MO,5,Springfield,31.0,Sally,NaN
4,Springfield,IL,1,NaN,NaN,NaN,NaN
5,Springfield,IL,1,NaN,30.0,NaN,4.0
6,Springfield,MO,5,NaN,NaN,NaN,NaN


In [67]:
# Perform the third merge: merge_outer_on
merge_outer_on = pd.merge(sales_and_managers, revenue_and_sales, on=["city", "state"], how = "outer")

# Print merge_outer_on
merge_outer_on

,city,state,units_x,branch,branch_id_x,manager,branch_id_y,revenue,units_y
0,Mendocino,CA,1,Mendocino,47.0,Brett,47.0,200.0,1
1,Denver,CO,4,Denver,20.0,Joel,20.0,83.0,4
2,Austin,TX,2,Austin,10.0,Charles,10.0,100.0,2
3,Springfield,MO,5,Springfield,31.0,Sally,NaN,NaN,5
4,Springfield,IL,1,NaN,NaN,NaN,30.0,4.0,1


Fantastic work! Notice how the default merge drops the Springfield rows, while the default outer merge includes them twice.



## 6. Using `merge_ordered()`
This exercise uses pre-loaded DataFrames `austin` and `houston` that contain weather data from the cities Austin and Houston respectively.

Weather conditions were recorded on separate days and you need to merge these two DataFrames together such that the dates are ordered. To do this, you'll use `pd.merge_ordered()`. After you're done, note the order of the rows before and after merging.

In [68]:
date = ["2016-01-01", "2016-02-08", "2016-01-17"]
ratings = ["Cloudy", "Cloudy", "Sunny"]

austin = pd.DataFrame({"date": date, "ratings": ratings})

austin

,date,ratings
0,2016-01-01,Cloudy
1,2016-02-08,Cloudy
2,2016-01-17,Sunny


In [69]:
date = ["2016-01-04", "2016-01-01", "2016-03-01"]
ratings = ["Rainy", "Cloudy", "Sunny"]

houston = pd.DataFrame({"date": date, "ratings": ratings})

houston

,date,ratings
0,2016-01-04,Rainy
1,2016-01-01,Cloudy
2,2016-03-01,Sunny


In [70]:
# Perform the first ordered merge: tx_weather
tx_weather = pd.merge_ordered(austin, houston)

# Print tx_weather
tx_weather

,date,ratings
0,2016-01-01,Cloudy
1,2016-01-04,Rainy
2,2016-01-17,Sunny
3,2016-02-08,Cloudy
4,2016-03-01,Sunny


In [71]:
# Perform the second ordered merge: tx_weather_suff
tx_weather_suff = pd.merge_ordered(austin, houston, on="date", suffixes=["_aus", "_hus"])

# Print tx_weather_suff
tx_weather_suff

,date,ratings_aus,ratings_hus
0,2016-01-01,Cloudy,Cloudy
1,2016-01-04,NaN,Rainy
2,2016-01-17,Sunny,NaN
3,2016-02-08,Cloudy,NaN
4,2016-03-01,NaN,Sunny


In [72]:
# Perform the third ordered merge: tx_weather_ffill
tx_weather_ffill = pd.merge_ordered(austin, houston, on="date", suffixes=["_aus", "_hus"], fill_method="ffill")

# Print tx_weather_ffill
tx_weather_ffill

,date,ratings_aus,ratings_hus
0,2016-01-01,Cloudy,Cloudy
1,2016-01-04,Cloudy,Rainy
2,2016-01-17,Sunny,Rainy
3,2016-02-08,Cloudy,Rainy
4,2016-03-01,Cloudy,Sunny


Well done! Notice how after using a fill method, there are no more `NaN` entries.



## 7. Using `merge_asof()`
Similar to `pd.merge_ordered()`, the `pd.merge_asof()` function will also merge values in order using the `on` column, but for each row in the left DataFrame, only rows from the right DataFrame whose `'on'` column values are less than the left value will be kept.

This function can be used to align disparate datetime frequencies without having to first resample.

Here, you'll merge monthly oil prices (US dollars) into a full automobile fuel efficiency dataset.

The datasets will align such that the first price of the year will be broadcast into the rows of the `automobiles` DataFrame. This is considered correct since by the start of any given year, most automobiles for that year will have already been manufactured.

You'll then inspect the merged DataFrame, resample by year and compute the mean `'Price'` and `'mpg'`. You should be able to see a trend in these two columns, that you can confirm by computing the Pearson correlation between resampled `'Price'` and `'mpg'`.

In [87]:
auto = pd.read_csv("data/automobiles.csv", parse_dates = ["yr"])
auto.tail()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
387,27.0,4,140.0,86,2790,15.6,1982-01-01,US,ford mustang gl
388,44.0,4,97.0,52,2130,24.6,1982-01-01,Europe,vw pickup
389,32.0,4,135.0,84,2295,11.6,1982-01-01,US,dodge rampage
390,28.0,4,120.0,79,2625,18.6,1982-01-01,US,ford ranger
391,31.0,4,119.0,82,2720,19.4,1982-01-01,US,chevy s-10


In [88]:
oil = pd.read_csv("data/oil_price.csv", parse_dates= ["Date"])
oil.tail()

,Date,Price
151,1982-08-01,33.95
152,1982-09-01,35.63
153,1982-10-01,35.68
154,1982-11-01,34.15
155,1982-12-01,31.72


In [89]:
# Merge auto and oil: merged
merged = pd.merge_asof(auto, oil, left_on="yr", right_on="Date")

# Print the tail of merged
merged.tail()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name,Date,Price
387,27.0,4,140.0,86,2790,15.6,1982-01-01,US,ford mustang gl,1982-01-01,33.85
388,44.0,4,97.0,52,2130,24.6,1982-01-01,Europe,vw pickup,1982-01-01,33.85
389,32.0,4,135.0,84,2295,11.6,1982-01-01,US,dodge rampage,1982-01-01,33.85
390,28.0,4,120.0,79,2625,18.6,1982-01-01,US,ford ranger,1982-01-01,33.85
391,31.0,4,119.0,82,2720,19.4,1982-01-01,US,chevy s-10,1982-01-01,33.85


In [91]:
# Resample merged: yearly
yearly = merged.resample("A", on="Date")[["mpg", "Price"]].mean()

# Print yearly
yearly

,mpg,Price
Date,,
1970-12-31,17.689655,3.35
1971-12-31,21.111111,3.56
1972-12-31,18.714286,3.56
1973-12-31,17.100000,3.56
1974-12-31,22.769231,10.11
1975-12-31,20.266667,11.16
1976-12-31,21.573529,11.16
1977-12-31,23.375000,13.90
1978-12-31,24.061111,14.85


In [92]:
# print yearly.corr()
yearly.corr()

,mpg,Price
mpg,1.000000,0.948677
Price,0.948677,1.000000


Great work! It looks like there is a strong correlation between miles per gallon and the price of oil!

